## Evaluate Approaches

This notebook verifies if identified subsets are valid.

An empty (or None) subset is automatically invalid

In [1]:
%load_ext autoreload
%autoreload 2


DATASET_NAME = "toxigen"
LABEL_SPACE = ["non-toxic", "toxic", "contradiction"]
MODEL_NAME = "deberta_large"
SEED = 42
POOLER = "mean_with_attention"
LAYER = 24

In [2]:
## Load Embeddings
from utils.io import (
    load_dataset_from_hf,
    load_labels_at_split,
    load_embeddings,
    load_wrapperbox
)
import numpy as np


train_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="train",
    pooler=POOLER,
    layer=LAYER
)

eval_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="eval",
    pooler=POOLER,
    layer=LAYER
)

test_embeddings = load_embeddings(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    split="test",
    pooler=POOLER,
    layer=LAYER
)

train_eval_embeddings = np.vstack([train_embeddings, eval_embeddings])


## Load Datasets and Labels
dataset = load_dataset_from_hf(dataset=DATASET_NAME)
train_labels = load_labels_at_split(dataset, "train")
eval_labels = load_labels_at_split(dataset, "eval")
train_eval_labels = np.concatenate([train_labels, eval_labels])
test_labels = load_labels_at_split(dataset, "test")

from datasets import DatasetDict, concatenate_datasets
train_eval_dataset = concatenate_datasets([dataset["train"], dataset["eval"]])
dataset_dict = DatasetDict(
    {"train": train_eval_dataset, "test": dataset["test"]}
)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/samsoup/.cache/huggingface/token
Login successful


/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/datasets/load.py:2516: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


In [6]:
## Do metrics for Logistic Regression baseline with purely class exclusion

import json
from pprint import pprint
from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions, compute_subset_metrics
import pickle

wrapper_name = "LogisticRegression"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

name = "LogisticRegression_baseline"
filename = f"{DATASET_NAME}_{MODEL_NAME}_{name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = compute_subset_metrics(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)


# Path to the output files
json_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_metrics.json'
pickle_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_is_valid.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(is_valid, pickle_file)

# Save the dictionary to a JSON file
with open(json_file_path, 'w', encoding ='utf8') as json_file:
    json.dump(metrics, json_file, indent=4)

  0%|          | 0/940 [00:00<?, ?it/s]


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: 1

In [8]:
## Do metrics for Yang fast

import json
from pprint import pprint
from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions, compute_subset_metrics
import pickle

wrapper_name = "LogisticRegression"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

name = "yang_fast"
filename = f"{DATASET_NAME}_{MODEL_NAME}_{name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = compute_subset_metrics(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)


# Path to the output files
json_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_metrics.json'
pickle_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_is_valid.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(is_valid, pickle_file)

# Save the dictionary to a JSON file
with open(json_file_path, 'w', encoding ='utf8') as json_file:
    json.dump(metrics, json_file, indent=4)

print(f"Saved to {pickle_file_path} and {json_file_path}")

100%|██████████| 940/940 [01:20<00:00, 11.73it/s]

5/931 identified subsets are valid
Overall validity is 5/940, or 0.53%
Precision validity is 5/931, or 0.54%
Identified 931/940 subsets.
Coverage: 99.04%
Median Valid Subset Sizes is 2430.0, out of 5 valid subsets
{'Coverage': 99.04,
 'Median Size': 2430.0,
 'Overall Validity': 0.53,
 'Precision Validity': 0.54}


In [11]:
with open(pickle_file_path, 'rb') as pickle_file:
    x = pickle.load(pickle_file)
x

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


In [ ]:
## Do metrics for Yang fast, with class exclusion as a fall back

import json
from pprint import pprint
from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions, compute_subset_metrics
import pickle

wrapper_name = "LogisticRegression"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

name = "yang_fast"
filename = f"{DATASET_NAME}_{MODEL_NAME}_{name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = compute_subset_metrics(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)


# Path to the output files
json_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_metrics.json'
pickle_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_is_valid.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(is_valid, pickle_file)

# Save the dictionary to a JSON file
with open(json_file_path, 'w', encoding ='utf8') as json_file:
    json.dump(metrics, json_file, indent=4)

In [3]:
## Do metrics for Yang slow

import json
from pprint import pprint
from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions, compute_subset_metrics
import pickle

wrapper_name = "LogisticRegression"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

name = "yang_slow"
filename = f"{DATASET_NAME}_{MODEL_NAME}_{name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = compute_subset_metrics(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)


# Path to the output files
json_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_metrics.json'
pickle_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_is_valid.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(is_valid, pickle_file)

# Save the dictionary to a JSON file
with open(json_file_path, 'w', encoding ='utf8') as json_file:
    json.dump(metrics, json_file, indent=4)

100%|██████████| 940/940 [00:04<00:00, 195.68it/s]

2/48 identified subsets are valid
Overall validity is 2/940, or 0.21%
Precision validity is 2/48, or 4.17%
Identified 48/940 subsets.
Coverage: 5.11%
Median Valid Subset Sizes is 90.5, out of 2 valid subsets
{'Coverage': 5.11,
 'Median Size': 90.5,
 'Overall Validity': 0.21,
 'Precision Validity': 4.17}


In [4]:
## Do metrics for KNN

import json
from pprint import pprint
from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions, compute_subset_metrics
import pickle

wrapper_name = "KNN"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

name = "KNN"
filename = f"{DATASET_NAME}_{MODEL_NAME}_{name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = compute_subset_metrics(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)


# Path to the output files
json_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_metrics.json'
pickle_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_is_valid.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(is_valid, pickle_file)

# Save the dictionary to a JSON file
with open(json_file_path, 'w', encoding ='utf8') as json_file:
    json.dump(metrics, json_file, indent=4)

/home/samsoup/anaconda3/envs/wrapperbox/lib/python3.9/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.1.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
100%|██████████| 940/940 [00:16<00:00, 57.18it/s]

940/940 identified subsets are valid
Overall validity is 940/940, or 100.0%
Precision validity is 940/940, or 100.0%
Identified 940/940 subsets.
Coverage: 100.0%
Median Valid Subset Sizes is 6144.0, out of 940 valid subsets
{'Coverage': 100.0,
 'Median Size': 6144.0,
 'Overall Validity': 100.0,
 'Precision Validity': 100.0}


In [7]:
## Do metrics for LGBM

import json
from pprint import pprint
from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions, compute_subset_metrics
import pickle

wrapper_name = "LGBM"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

name = "LGBM"
filename = f"{DATASET_NAME}_{MODEL_NAME}_{name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = compute_subset_metrics(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)


# Path to the output files
json_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_metrics.json'
pickle_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_is_valid.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(is_valid, pickle_file)

# Save the dictionary to a JSON file
with open(json_file_path, 'w', encoding ='utf8') as json_file:
    json.dump(metrics, json_file, indent=4)

  0%|          | 0/940 [00:00<?, ?it/s]

[LightGBM] [Info] Number of positive: 1414, number of negative: 6760
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.086979 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 8174, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.172988 -> initscore=-1.564600
[LightGBM] [Info] Start training from score -1.564600


  2%|▏         | 22/940 [00:00<00:28, 32.58it/s]

[LightGBM] [Info] Number of positive: 2172, number of negative: 6776
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.074756 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 8948, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242736 -> initscore=-1.137739
[LightGBM] [Info] Start training from score -1.137739


  3%|▎         | 26/940 [00:01<01:21, 11.18it/s]

...

[LightGBM] [Info] Number of positive: 2172, number of negative: 6779
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.091624 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 8951, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242654 -> initscore=-1.138181
[LightGBM] [Info] Start training from score -1.138181


 99%|█████████▉| 934/940 [01:11<00:00, 13.23it/s]

[LightGBM] [Info] Number of positive: 2173, number of negative: 6779
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.084401 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 8952, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242739 -> initscore=-1.137721
[LightGBM] [Info] Start training from score -1.137721


100%|██████████| 940/940 [01:12<00:00, 12.95it/s]

[LightGBM] [Info] Number of positive: 2174, number of negative: 6782
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.092732 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 261120
[LightGBM] [Info] Number of data points in the train set: 8956, number of used features: 1024
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.242742 -> initscore=-1.137703
[LightGBM] [Info] Start training from score -1.137703
126/126 identified subsets are valid
Overall validity is 126/940, or 13.4%
Precision validity is 126/126, or 100.0%
Identified 126/940 subsets.
Coverage: 13.4%
Median Valid Subset Sizes is 4.0, out of 126 valid subsets
{'Coverage': 13.4,
 'Median Size': 4.0,
 'Overall Validity': 13.4,
 'Precision Validity': 100.0}


In [5]:
## Do metrics for LMeans

import json
from pprint import pprint
from MinimalSubsetToFlipPredictions.evaluate import evaluate_predictions, compute_subset_metrics
import pickle

wrapper_name = "LMeans"
clf = load_wrapperbox(
    dataset=DATASET_NAME,
    model=MODEL_NAME,
    seed=SEED,
    pooler=POOLER,
    wrapperbox=wrapper_name
)

name = "LMeans"
filename = f"{DATASET_NAME}_{MODEL_NAME}_{name}.pickle"
with open(filename, 'rb') as handle:
    flip_list = pickle.load(handle)

ex_indices_to_check = np.arange(len(flip_list))
is_valid = evaluate_predictions(
    clf=clf,
    flip_list=flip_list,
    train_embeddings=train_eval_embeddings,
    train_labels=train_eval_labels,
    test_embeddings=test_embeddings,
    ex_indices_to_check=ex_indices_to_check,
)

metrics = compute_subset_metrics(
    flip_list=flip_list,
    is_valid=is_valid
)

pprint(metrics)


# Path to the output files
json_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_metrics.json'
pickle_file_path = f'{DATASET_NAME}_{MODEL_NAME}_{name}_is_valid.pickle'

# Save the dictionary to a pickle file
with open(pickle_file_path, 'wb') as pickle_file:
    pickle.dump(is_valid, pickle_file)

# Save the dictionary to a JSON file
with open(json_file_path, 'w', encoding ='utf8') as json_file:
    json.dump(metrics, json_file, indent=4)

100%|██████████| 940/940 [01:09<00:00, 13.46it/s]

940/940 identified subsets are valid
Overall validity is 940/940, or 100.0%
Precision validity is 940/940, or 100.0%
Identified 940/940 subsets.
Coverage: 100.0%
Median Valid Subset Sizes is 6339.0, out of 940 valid subsets
{'Coverage': 100.0,
 'Median Size': 6339.0,
 'Overall Validity': 100.0,
 'Precision Validity': 100.0}
